#### CRUD operations

Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

**TODO 1**: Take the dict created in the TODO 4 in chapter II and save it in the collection "CRUD_exercise".

**TODO 2**: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

**TODO 3**: Insert the dict created in the TODO 6 Chapter II in the example collection.

**TODO 4**: Get documents where authors key exist in the collection "CRUD_exercise".

**TODO 5**: Change the documents where x = 4 to x = 1.

**TODO 6**: Find documents where author is not_mike and set author as real_mike.

**TODO 7**: Delete documents where author is real_mike.

#### Managing DB

**TODO 8**: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

**TODO 9**: create a random collection in a random db and put the new collection in the tutorial DB

**TODO 10**: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

#### Real world problems

**TODO 11**:  Use the oaipmh and api code get papers "econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "econ".

**TODO 12**: Do the same as TODO 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.

**TODO 13**: Download a random image and store it in a collection.

**TODO 14**: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)

**TODO 15**: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.

**TODO 16**: Download a [sound sample](https://freesound.org/browse/). Try to store it in MongoDB 

**TODO 17**: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ?

In [2]:
import pymongo
import tqdm
import numpy as np

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["Todo"]
collection = db["CRUD_exercise"]

In [ ]:

#1
lecun_paper = {
    "title": "Deep Learning",
    "authors": {
        "Yann LeCun": {"affiliations": ["Facebook AI Research", "New York University"]},
        "Yoshua Bengio": {"affiliations": ["Department of Computer Science and Operations Research Université de Montréal"]},
        "Geoffrey Hinton": {"affiliations": ["Google", "Department of Computer Science, University of Toronto"]}
    }
}
goodfellow_paper = {
    "title": "Generative Adversarial Nets",
    "authors": {
        "Ian Goodfellow": {"affiliations": ["Universite de Montreal"]},
        "Jean Pouget-Abadie": {"affiliations": ["Ecole Polytechnique"]},
        "Mehdi Mirza": {"affiliations": ["Université de Montréal"]},
        "Bing Xu": {"affiliations": ["Université de Montréal"]},
        "David Warde-Farley": {"affiliations": ["Université de Montréal"]},
        "Sherjil Ozair": {"affiliations": ["Indian Institute of Technology Delhi"]},
        "Aaron Courville": {"affiliations": ["Université de Montréal"]},
        "Yoshua Bengio": {"affiliations": ["CIFAR Senior Fellow"]}
    }
}
papers_dict = {
    "LeCun et al.": lecun_paper,
    "Goodfellow et al.": goodfellow_paper
}

collection.insert_one(papers_dict)

#2
doc1 = {"x": 1, "value": 1}
doc2 = {"x": 1, "value": 1}
doc3 = {"x": 1, "value": 1}

collection.insert_one(doc1)
collection.insert_one(doc2)
collection.insert_one(doc3)

collection.delete_one({"x": 1})

collection.update_many({"x": 1}, {"$set": {"x": 4}})

#3

import lxml.etree

xml_file2 = "/Users/lyna/Documents/GitHub/NoSQL/data/Chap2/xml_file2.nxml"
root = lxml.etree.parse(xml_file2)

date = root.xpath("//date//text()")
hour = root.xpath("//hour//text()")
to = root.xpath("//to//text()")
from_ = root.xpath("//from//text()")
body = root.xpath("//body//text()")

info_dict = {
    "date": date,
    "hour": hour,
    "to": to,
    "from": from_,
    "body": body
}
collection.insert_one(info_dict)

#4

docs = collection.find({"authors":{"$exists":1}})
list(docs)

#5
collection.update_many({"x": 4}, {"$set": {"x": 1}})

#6
collection.update_many({"author": "not_mike"}, {"$set": {"author": "real_mike"}})

#7
collection.delete_many({"author": "real_mike"})

#8
import pymongo
client = pymongo.MongoClient('localhost', 27017)
mydb = client["Todo"]
collection = mydb["CRUD_exercise_benchmark"]

import numpy as np

values = np.random.rand(500000, 4)
docs = [{"id": i, "values": values[i//2].tolist()} for i in range(0, 1000000, 2)]

collection.insert_many(docs)

import time

# No index
start_time = time.time()
docs = collection.find({"id": 100000})
end_time = time.time()
print("Temps sans index: ", end_time - start_time)

# Index
start_time = time.time()
docs = collection.find({"id": 100000}).hint("id_1")
end_time = time.time()
print("Temps avec index: ", end_time - start_time)

#9
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
random_db = client["random_db"]
random_collection = random_db["random_collection"]

doc = {"key": "value"}
random_collection.insert_one(doc)

tutorial_db = client["tutorial"]
tutorial_collection = tutorial_db["tutorial_collection"]
tutorial_collection.insert_many(random_collection.find())

#10
# Inner join returns only the matching documents from both collections whereas outer join returns all documents 
# from both collections, including non-matching documents as null. The query seen during the course was an inner join.


In [1]:
#11


#12


#13


#14


#15


#16


#17

from pymongo import MongoClient
from random import randint

client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['year']

batch_size = 10000
total_records = 30000000
records = []

for i in range(total_records):
    year = randint(2000, 2020)
    record = {'year': year}
    records.append(record)
    if len(records) == batch_size:
        collection.insert_many(records)
        records = []

if records:
    collection.insert_many(records)


start_time = time.time()
docs = collection.count_documents({"year": 2000})
end_time = time.time()
print('Nombre de documents avec une année égale à 2000 :', docs)
print("Temps sans index: ", end_time - start_time)

collection.create_index([("year", 1)])
start_time = time.time()
docs = collection.count_documents({"year": 2000})
end_time = time.time()
print('Nombre de documents avec une année égale à 2000 :', docs)
print("Temps avec index: ", end_time - start_time)
